<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/TME522.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data description

We consider the Books dataset which describes books and users rating these books. The schema of this dataset is given as follows:

* `Users (userid: Number, country: Text, age: Number)` 
* `Books (bookid: Number, titlewords: Number, authorwords: Number, year: Number, publisher: Number)`
* `Ratings (userid: Number, bookid: Number, rating: Number)`

In the Ratings table, userid and bookid refer to Users and Books, respectively.

In [ ]:
users.printSchema()
books.printSchema()
ratings.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- age: integer (nullable = true)

root
 |-- bookid: integer (nullable = true)
 |-- titlewords: integer (nullable = true)
 |-- authorwords: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- publisher: integer (nullable = true)

root
 |-- userid: integer (nullable = true)
 |-- bookid: integer (nullable = true)
 |-- rating: integer (nullable = true)



### Simple queries

#### s0) Ids of users (column userid) from France. Note that country names are in lower case

In [ ]:
s0 = users.select('userid').where(users.country == 'france')
s0.count()

#### s1) Ids of books (column bookid) whose publication year is 2000

In [ ]:
s1 = books.select('bookid').where(books.year>2000)
s1.count()

#### s2) Ids of books rated above 3 (>3)

In [ ]:
s2 =  ratings.select('bookid').where(ratings.rating>3)
s2.count()

### Collecting basic statistics

#### Total number of distinct users

In [ ]:
users.select('userid').distinct().count()

#### Total number of distinct  books

In [ ]:
books.select('bookid').distinct().count()

### Aggregation queries

#### q1) Number of users per country, sorted in descending order of this number

In [ ]:
q1 = users.groupBy('country').count().orderBy(col('count').desc())
q1.show()

##### Country who has the highest number of users, together with this number. Assume that only one country has this number.

In [ ]:
q11 = q1.limit(1).select("country")
q11.show()

##### Year with the highest number of edited books, together with this number. Assume that only one year has this number.

In [ ]:
q12 = books.groupBy('year').count().orderBy(col('count').desc()).limit(1).select('year')
q12.show()


#### q2) Publishers with more than ten (10) edited books, in total

In [ ]:
q2 = books.groupby('publisher').count().orderBy(col('count').desc()).where(col('count')>10)
q2.count()

#### q3) Publishers with more than five (5) edited books for each year in which they have published a book

In [ ]:
q3 = books.select('publisher')\
    .subtract(books.groupBy(col('publisher'), col("year"))\
    .count().where(col("count")<5)\
    .select('publisher'))
q3.count()

#### q4) The average rating per book
pour faire une aggrégation il faut forcement faire un groupby avant

In [ ]:
q4 = ratings.groupBy('bookid').avg('rating')
q4.show()

### Join queries

#### q5) The publishers of books rated by users living in France
1)jointure entre user et rating sur userid living in france
2) jointure avec book sur bookid 
3)selectionne publisher 

In [ ]:
q5 = users.join(ratings, 'userid').where(col('country') == 'france')\
     .join(books, 'bookid')\
     .select('publisher').distinct()
q5.count()

#### q6) The publishers of books which were never rated by users living in France

In [ ]:
q6=books.select('publisher').subtract(q5)
q6.count()


### Queries using built-in functions

The Spark API contains many useful built-in functions that can be directly invoked on a dataframe. These are documented:
* https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions

The goal of this section is two use some of these functions to compute the Jaccard Similarity between users based on the books they rated.
To do so, we need to collect, for each pair of users (u1,u2), the sets of books they have rated, eg. [b1, ..., bn] for u1 and [b'1, ...,b'm] for u2, than apply the similarity formulae explained in https://en.wikipedia.org/wiki/Jaccard_index, that is, dividing the intersection of the sets of books by the union of these sets.

$$sim(u1,u2) = |([b_1, ..., b_n] \cap [b'_1, ...,b'_m]) /  ([b_1, ..., b_n] \cup [b'_1, ...,b'_m])|$$

**Note. Due to the potential high cost for computing the cross-product between all users, we restrict to users of France.**

#### create a df with ratings restricted to users of 'france' .

In [ ]:
users_fr = users.where("country = 'france'")
users_fr.count()

In [ ]:
ratings_fr = ratings.join(users_fr,"userid")
ratings_fr.count()

**In this part, users always refer to users of France **

#### create a dataframe obtained by collecting, for each user, the set of rated books. 
Hint. group bookids per user than use a built-in function that creates an array from the grouped bookids (examine the schema)

In [ ]:
users_books = ratings_fr.groupBy('userid')\
              .agg(collect_list('bookid')\
              .alias('set_bookid'))
users_books.show()

#### create a dataframe containing pairs of distinct users with their rated books.
Hint. You need to rename the dataframe columns.

In [ ]:
pair_users_books = users_books.crossJoin(users_books.select(col('userid').alias('userid2'), col('set_bookid').alias('set_bookid2')))\
                  .where((col('userid') != col('userid2')) & (col('userid') < col('userid2')))\
                  .select(struct(col('userid'), col('userid2')).alias('usersid'), col('set_bookid'), col('set_bookid2'))

pair_users_books.show()

#### compute the Jaccard similarity and leave only pairs of books with a non-zero similarity

In [ ]:
from pyspark.sql.functions import array_intersect, array_union, size

In [ ]:
jaccard_sim = pair_users_books\
              .withColumn('sim', size(array_intersect(col('set_bookid'),col('set_bookid2')))/size(array_union(col('set_bookid'), col('set_bookid2'))))\
              .where(col('sim') > 0)\
              .select('usersid.userid', 'usersid.userid2', 'sim')\
              .orderBy(col('sim').desc())



In [ ]:
jaccard_sim.show()

In [ ]:
jaccard_sim.count()

### Queries with User-defined functions

Spark allows users to define specific functions called `User-Defined Functions`. 
We illustrate this concept with the following example: 

consider that we need to return the number of characters of the `country` column. To do so, we define a function called `slen` which, given a string `s` as  input returns its length computed by the string function `len(s)`.
The `udf` will be invoked on a dataframe by specifying the column(s) on which it is applied.

There are different ways to define a `udf`:
* using the `udf` class and registering it using the `register` method of the `udf` class, or
* by preceding the function siganture with `@udf('type')` where `type` is the return type of the function

We will use the second option which is syntactically simpler.

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
@udf('integer')
def slen(s):
  return len(s)

In [ ]:
len_country = users.withColumn("length",slen("country"))
len_country.show()

#### Mapping ratings to categories

We would like to add a textual representation of ratings such that:
* rating <1 is converted to 'bad'
* 1 <= rating <2 is converted to 'average'
* 2 <= rating <3 is converted to 'good'
* 3 <= rating is converted to 'excellent'

#### Complete the stub of `convert_rating(note)` which maps an integer to a string based on the previous rules.

In [ ]:
@udf('string')
def convert_rating(note):
   if note<=1:
    return "bad"
   elif note <=2:
        return "average"
   elif note <=3:
         return "good"
   else:
         return "excellent"

#### Using  `convert_rating` map each `rating` to its associated category
remplace colomne rating par convert(rating)

In [ ]:
text_ratings = ratings.withColumn('rating', convert_rating(col('rating')))
text_ratings.show()

### Queries with vectorized User-defined functions

Adopt another strategy by defining a Panda UDF for mapping rating to categories

In [ ]:
@pandas_udf('string')
def convert_rating2(notes):
    return notes.map(lambda note: "bad" if note<=1 else 
                                  "average" if note <=2 else 
                                  "good" if note <=3 else "excellent")

text_ratings = ratings.withColumn('rating', convert_rating2("rating"))
text_ratings.show()

## END